In [ ]:
!pip install -q biopython

%load_ext autoreload
%autoreload 2

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
DATA_DIR = '/content/drive/MyDrive/data/uci/'
LOG_DIR = '/content/drive/MyDrive/dna-nn/results/'

In [ ]:
!cp -r '/content/drive/MyDrive/dna-nn/dna_nn' .
!ls dna_nn

dataset.py  download.py  load.py  model.py  __pycache__


In [ ]:
import gc

import matplotlib.pyplot as plt
import pandas as pd
from tensorflow import keras

from dna_nn.dataset import splice
from dna_nn.model import models, evaluate

In [ ]:
x_shape, x_train, x_test, y_train, y_test = splice(DATA_DIR + 'splice.data')

In [ ]:
models

{'cnn_deepdbp': <function dna_nn.model.cnn_deepdbp>,
 'cnn_nguyen_2_conv2d': <function dna_nn.model.cnn_nguyen_2_conv2d>,
 'cnn_nguyen_conv1d_2_conv2d': <function dna_nn.model.cnn_nguyen_conv1d_2_conv2d>,
 'cnn_zeng_2_conv2d': <function dna_nn.model.cnn_zeng_2_conv2d>,
 'cnn_zeng_3_conv2d': <function dna_nn.model.cnn_zeng_3_conv2d>,
 'cnn_zeng_4_conv2d': <function dna_nn.model.cnn_zeng_4_conv2d>}

In [ ]:
key = 'cnn_nguyen_conv1d_2_conv2d'
dataset = 'splice'

keras.backend.clear_session()
model = models[key](x_shape, 3)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')
model.summary()

something
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 58, 58, 16)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 29, 29, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 27, 16)        2320      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 32)        4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 800)      

In [ ]:
csv_path = LOG_DIR + f'{key}-{dataset}-dynamics.csv'
model_path = LOG_DIR + f'{key}-{dataset}.h5'

callbacks = [
    keras.callbacks.CSVLogger(csv_path),
    keras.callbacks.LambdaCallback(
        on_epoch_end=lambda epoch, logs: gc.collect(),
        # on_train_end=lambda logs: model.save(model_path)
    ),
    keras.callbacks.ModelCheckpoint(model_path, save_best_only=True)
]

history = model.fit(x_train, y_train, epochs=50, validation_split=0.15,
                    callbacks=callbacks, verbose=2)

Epoch 1/50
64/64 - 5s - loss: 0.9806 - accuracy: 0.5229 - val_loss: 0.7850 - val_accuracy: 0.7131
Epoch 2/50
64/64 - 4s - loss: 0.6311 - accuracy: 0.7295 - val_loss: 0.4465 - val_accuracy: 0.8440
Epoch 3/50
64/64 - 4s - loss: 0.4502 - accuracy: 0.8136 - val_loss: 0.3151 - val_accuracy: 0.8858
Epoch 4/50
64/64 - 4s - loss: 0.3886 - accuracy: 0.8431 - val_loss: 0.2838 - val_accuracy: 0.9081
Epoch 5/50
64/64 - 4s - loss: 0.3250 - accuracy: 0.8775 - val_loss: 0.2684 - val_accuracy: 0.8997
Epoch 6/50
64/64 - 4s - loss: 0.2916 - accuracy: 0.8913 - val_loss: 0.2173 - val_accuracy: 0.9248
Epoch 7/50
64/64 - 4s - loss: 0.2291 - accuracy: 0.9213 - val_loss: 0.2016 - val_accuracy: 0.9276
Epoch 8/50
64/64 - 4s - loss: 0.2061 - accuracy: 0.9336 - val_loss: 0.2054 - val_accuracy: 0.9387
Epoch 9/50
64/64 - 4s - loss: 0.1892 - accuracy: 0.9380 - val_loss: 0.2132 - val_accuracy: 0.9387
Epoch 10/50
64/64 - 4s - loss: 0.1656 - accuracy: 0.9439 - val_loss: 0.1964 - val_accuracy: 0.9359
Epoch 11/50
64/64 -

In [ ]:
model = keras.models.load_model(model_path)
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=3)

y_score = model.predict(x_test)
y_true = [int(y) for y in y_test]
evaluate(model, history, test_accuracy, y_score, y_true, LOG_DIR, key, dataset, 3)